<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/MDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime 
from datetime import timedelta
import pickle
from google.colab import drive

drive.mount('/content/drive')
output_path = '/content/drive/MyDrive/K-project/output/'
data_path = '/content/drive/MyDrive/K-project/'
# pickle 경로를 설정하기 위해

In [ ]:
# 한국시간 설정
import pytz

KST = pytz.timezone('Asia/Seoul')
today = str(datetime.now(KST).strftime("%Y%m%d"))
print(today)

In [ ]:
## Load pickle - fullcode 목록
with open(data_path + 'full_code.pkl',"rb") as f: 
    fullcode = pickle.load(f)
fullcode

In [ ]:
#상장폐지 상태 제외시키기.
fullcode_filter = fullcode[fullcode['status'] != '상장폐지']
fullcode_filter

In [ ]:
fullcode_list = list(fullcode_filter['full_code'])
print(fullcode_list[:10])
# fullcode열을 list로 변경 후 확인

In [ ]:
url2 = 'http://data.krx.co.kr/comm/bldAttendant/getJsonData.cmd'
headers = {'User-Agent': 'Chrome/78.0.3904.87 Safari/537.36',}

strtDd = '19000101'
endDd = today

for i in fullcode_list[:2]: # 테스트를 위해 2개만 실행 (두 종목만 출력됨)
    data2 = { 'bld': 'dbms/MDC/STAT/issue/MDCSTAT23902',
    'isuCd': i,
    'isuCd2': '',
    'strtDd': strtDd,
    'endDd': endDd,
    'share': '1',
    'money': '1',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data2, headers)
    j2 = json.loads(r2.text)
    df2=pd.DataFrame(j2['output'])
    # print(df2)
    # print('------------------------------------------------')

    data3 = { 'bld': 'dbms/MDC/STAT/standard/MDCSTAT03502',
    'isuCd': i,
    'isuCd2': i,
    'strtDd': strtDd,
    'endDd': endDd,
    'searchType': '2',
    'mktId': 'ALL',
    'csvxls_isNo': 'false',}

    r2 = requests.post(url2, data3, headers)
    j2 = json.loads(r2.text)
    df3=pd.DataFrame(j2['output'])
    # print(df3)
    # print('------------------------------------------------')

    df2_ = df2[['TRD_DD', 'ISU_CD', 'ISU_NM', 'TDD_CLSPRC', 'TDD_OPNPRC', 'TDD_HGPRC', 'TDD_LWPRC', 'MKTCAP']]
    df3_ = df3[['TRD_DD', 'EPS', 'PER', 'BPS', 'PBR', 'DPS', 'DVD_YLD']]

    result = pd.merge(df2_, df3_)
    print(result)
    print('------------------------------------------------')

In [ ]:
!wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 2>&1 > /dev/null
!tar xvzf ta-lib-0.4.0-src.tar.gz 2>&1 > /dev/null

import os

os.chdir('ta-lib')  

!./configure --prefix=/usr 2>&1 > /dev/null
!make 2>&1 > /dev/null
!make install 2>&1 > /dev/null

os.chdir('../')

!pip install TA-Lib 2>&1 > /dev/null

In [ ]:
result

In [ ]:
result = result.sort_values(by=['TRD_DD'])
result.set_index('TRD_DD',drop=True,inplace=True)
result.reset_index('TRD_DD',drop=False,inplace=True)
result

In [ ]:
result['TDD_CLSPRC'] = result['TDD_CLSPRC'].str.replace(',','').astype('float')
result['TDD_OPNPRC'] = result['TDD_OPNPRC'].str.replace(',','').astype('float')
result['TDD_HGPRC'] = result['TDD_HGPRC'].str.replace(',','').astype('float')
result['TDD_LWPRC'] = result['TDD_LWPRC'].str.replace(',','').astype('float')
result['MKTCAP'] = result['MKTCAP'].str.replace(',','').astype('float')
result['BPS'] = result['BPS'].str.replace(',','').astype('float')
result

In [ ]:
result = pd.DataFrame({tic: result['TDD_CLSPRC'] for tic, data in result.items()})
result = result.fillna(method='ffill')

In [ ]:
result

In [ ]:
#MDD구하기
import matplotlib.pyplot as plt

def MDD(data) :
    mdd = 0
    for i in range(1,len(data)-1):
        maxv = max(data[:i])
        calcMdd = (maxv - min(data[i:])) / maxv
        if calcMdd > mdd:
            mdd = calcMdd
    return mdd

if __name__ == "__main__" :
    sample = result['TDD_CLSPRC']

    # 그래프
    plt.plot(sample)
    plt.grid()
    plt.show()

    # MDD 구하기
    print(MDD(sample))